In [1]:
import numpy as np
import cv2
import torch

### Select device

In [2]:
device = torch.device("cuda")

### Create default or your own generator and EMA generator

In [9]:
from generator import define_G

def make_generator():
    gen = define_G(n_downsampling=4, n_resnet_blocks=4).to(device)
    return gen
generator = make_generator()
generator_ema = make_generator()

# Initilalize generators with equal parameters
with torch.no_grad():
    for (gp, ep) in zip(generator.parameters(), generator_ema.parameters()):
        ep.data = gp.data.detach()

Generator(
  (model): Sequential(
    (0): ConvBlock(
      (pad): ReflectionPad2d((3, 3, 3, 3))
      (norm): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (activation): ReLU(inplace=True)
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    )
    (1): ConvBlock(
      (pad): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      (norm): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (activation): ReLU(inplace=True)
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
    )
    (2): ConvBlock(
      (pad): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      (norm): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (activation): ReLU(inplace=True)
      (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
    )
    (3): ConvBlock(
      (pad): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      (norm): InstanceNorm2d(512, eps=1e-05, mo

### Use the default or your own discriminator

In [10]:
from discriminator import define_D

discriminator = define_D(input_nc = 3 + 3, ndf = 64, n_layers_D = 3, num_D = 3, norm="instance", getIntermFeat=True).to(device)

MultiscaleDiscriminator(
  (scale0_layer0): Sequential(
    (0): Conv2d(6, 64, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (scale0_layer1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
    (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (scale0_layer2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
    (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (scale0_layer3): Sequential(
    (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
    (1): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (scale0_layer4): Sequential(
   

In [11]:
import losses
criterionGAN = losses.GANLoss(use_lsgan=True).to(device)
criterionFeat = torch.nn.L1Loss().to(device)
criterionVGG = losses.VGGLoss().to(device)
# import Loss_Function
# criterionGAN = Loss_Function.GANLoss(use_lsgan=True).to(device)
# criterionFeat = torch.nn.L1Loss().to(device)
# criterionVGG = Loss_Function.VGGLoss().to(device)
# criterionGrad = Loss_Function.DiscriminatorGradientPenaltyLoss().to(device)

In [12]:
from replay_pool import ReplayPool
replay_pool = ReplayPool(10)

In [13]:
G_optim = torch.optim.AdamW(generator.parameters(), lr=1e-4)
D_optim = torch.optim.AdamW(discriminator.parameters(), lr=1e-4)

### Modify the dataset class for your requirements

In [5]:
from torchvision import transforms
from random import uniform, randint
from glob import glob
import os

class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, images_dir):

        self.to_tensor = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        self.imagesDir = images_dir
        self.images = glob(images_dir + "*.jpg")

        
    def __getitem__(self, idx):
        from random import random, randint, uniform
        f_name = self.images[idx]

        pair = cv2.imread(f_name, 1)
        pair = cv2.cvtColor(pair, cv2.COLOR_BGR2RGB)
        mid = pair.shape[1]//2
        dst = pair[:, :mid]
        src = pair[:, mid:]
        
        w = 256
        h = 256

        if random() < 0.5:
            src = np.fliplr(src)
            dst = np.fliplr(dst)

        src_tensor = self.to_tensor(src.copy())
        dst_tensor = self.to_tensor(dst.copy())
        return src_tensor, dst_tensor
        
    def __len__(self):
        return len(self.images)
        

In [6]:
batch_size = 1
train_dataset = Dataset("./train_small/")
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, num_workers=4, shuffle=True)
test_dataset = Dataset("./val_small/")
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size, num_workers=4, shuffle=True)


In [10]:
targets, data = next(iter(train_loader))

In [11]:
data.shape

torch.Size([1, 3, 1024, 1024])

In [12]:
checkpoint_dir = "./checkpoints/MaskGAN/"
images_output_dir = os.path.join(checkpoint_dir, "images")
if not os.path.exists(images_output_dir):
    os.makedirs(images_output_dir)

In [15]:
test_output_dir = os.path.join(checkpoint_dir, "test")
if not os.path.exists(test_output_dir):
    os.makedirs(test_output_dir)

In [33]:
import os
def test(epoch, iteration):
    with torch.no_grad():
        target, data = next(iter(test_loader))
        new_target, new_data = next(iter(test_loader))
#         data, target = next(iter(test_loader)) 
        new_data = new_data.to(device)
        generator_ema.eval()
        target = target.to(device)
        out = generator_ema(new_data, new_data, target)
        generator_ema.train()
        matrix = []
        pairs = torch.cat([new_data, out, target.to(device)], -1)
        for idx in range(new_data.shape[0]):
            img = 255*(pairs[idx] + 1)/2
            img = img.cpu().permute(1, 2, 0).clip(0, 255).numpy().astype(np.uint8)
            matrix.append(img)
        matrix = np.vstack(matrix)
        matrix = cv2.cvtColor(matrix, cv2.COLOR_RGB2BGR)
        out_file = os.path.join(test_output_dir, f"{epoch}_{iteration}.jpg")
        cv2.imwrite(out_file, matrix)
    

In [14]:
from moving_average import moving_average
from tqdm import tqdm

def process_loss(log, losses):
    loss = 0
    for k in losses:
        if k not in log:
            log[k] = 0
        log[k] += losses[k].item()
        loss = loss + losses[k]
    return loss

def calc_G_losses(data, target):
    fake = generator(data, data, target)
    loss_vgg = 1*criterionVGG(fake, target)
    pred_fake = discriminator(torch.cat([data, fake], axis=1))
    loss_adv = 1*criterionGAN(pred_fake, 1)

    with torch.no_grad():
        pred_true = discriminator(torch.cat([data, target], axis=1))

    loss_adv_feat = 0
    adv_feats_count = 0        
    for d_fake_out, d_true_out in zip(pred_fake, pred_true):
        for l_fake, l_true in zip(d_fake_out[: -1], d_true_out[: -1]):
            loss_adv_feat = loss_adv_feat + criterionFeat(l_fake, l_true)
            adv_feats_count += 1
    loss_adv_feat = 1*(4/adv_feats_count)*loss_adv_feat
    lambda_feat = 10
    return {"G_vgg": loss_vgg, "G_adv": loss_adv, "G_adv_feat": lambda_feat*loss_adv_feat}

def calc_D_losses(data, target):
    with torch.no_grad():
        gen_out = generator(data, data, target)
        fake = replay_pool.query({"input": data.detach(), "output": gen_out.detach()})
    pred_true = discriminator(torch.cat([data, target], axis=1))
    loss_true = criterionGAN(pred_true, 1)
    pred_fake = discriminator(torch.cat([fake["input"], fake["output"]], axis=1))
    loss_false = criterionGAN(pred_fake, 0)
    return {"D_true": loss_true, "D_false": loss_false}

def train(epoch):
    
    print(f"Training epoch {epoch}...")
    generator.train()
    discriminator.train()
    N = 0
    log = {}
    pbar = tqdm(train_loader)
    for target, data in pbar:
        with torch.no_grad():
            data = data.to(device)
            target = target.to(device)
        
        G_optim.zero_grad()
        generator.requires_grad_(True)
        discriminator.requires_grad_(False)
        G_losses = calc_G_losses(data, target)
        G_loss = process_loss(log, G_losses)
        G_loss.backward()
        del G_losses
        G_optim.step()
        moving_average(generator, generator_ema)
        
        D_optim.zero_grad()
        generator.requires_grad_(False)
        discriminator.requires_grad_(True)
        D_losses = calc_D_losses(data, target)
        D_loss = process_loss(log, D_losses)
        D_loss.backward()
        del D_losses
        D_optim.step()
        
        txt = ""
        N += 1
        if (N%100 == 0) or (N + 1 >= len(train_loader)):
            for i in range(3):
                test(epoch, N + i)
        for k in log:
            txt += f"{k}: {log[k]/N:.3e} "
        pbar.set_description(txt)
        
        if (N%1000 == 0) or (N + 1 >= len(train_loader)):
            import datetime
            out_file = f"epoch_{epoch}_{datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}.pt"
            out_file = os.path.join(checkpoint_dir, out_file)
            torch.save({"G": generator_ema.state_dict(), "D": discriminator.state_dict()}, out_file)
            print(f"Saved to {out_file}")

### Use the next lines for restoring from the checkpoint

In [20]:
!pwd

/projectnb/dl523/students/haolinye/Project


In [29]:
ckpt_file = "./checkpoints/MaskGAN/epoch_50_2023-04-25 23:25.pt"
ckpt = torch.load(ckpt_file)
ckpt

{'G': OrderedDict([('model.0.conv.weight',
               tensor([[[[ 2.4573e-02, -2.4039e-03, -1.4262e-02,  ...,  3.1084e-02,
                           2.1297e-02,  1.0411e-02],
                         [ 5.3037e-03, -1.0415e-02,  2.3375e-03,  ..., -2.6337e-03,
                          -2.2887e-02, -1.5242e-02],
                         [ 1.1056e-02, -1.4413e-04, -1.8611e-03,  ..., -1.2506e-02,
                           2.3236e-02, -3.6735e-02],
                         ...,
                         [ 7.8401e-03,  1.2465e-02, -4.3654e-03,  ..., -7.3282e-03,
                           2.7169e-02, -3.6700e-02],
                         [-2.3865e-02,  3.5172e-02, -1.1453e-02,  ..., -2.0735e-02,
                           3.1891e-02, -5.4559e-03],
                         [ 2.1595e-03, -6.6529e-03, -2.6121e-02,  ...,  8.3571e-03,
                           3.2547e-03, -3.7791e-02]],
               
                        [[ 7.6092e-03, -2.9651e-02,  2.1683e-02,  ...,  9.6148e-03,
    

In [30]:
generator.load_state_dict(ckpt["G"])
generator_ema.load_state_dict(ckpt["G"])
discriminator.load_state_dict(ckpt["D"])

<All keys matched successfully>

In [34]:
test(0, 0)

### Start

In [ ]:
for epoch in range(0, 1000):
    train(epoch)

  0%|          | 0/401 [00:00<?, ?it/s]

Training epoch 0...


/share/pkg.7/pytorch/1.9.0/install/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
G_vgg: 5.372e-01 G_adv: 1.424e+00 G_adv_feat: 7.573e+00 D_true: 9.617e-01 D_false: 8.759e-01 : 100%|█████████▉| 400/401 [09:14<00:02,  2.19s/it]

Saved to ./checkpoints/MaskGAN/epoch_0_2023-04-25 15:39.pt


G_vgg: 5.371e-01 G_adv: 1.424e+00 G_adv_feat: 7.568e+00 D_true: 9.619e-01 D_false: 8.746e-01 : 100%|██████████| 401/401 [09:18<00:00,  1.39s/it]
  0%|          | 0/401 [00:00<?, ?it/s]

...

G_vgg: 4.645e-01 G_adv: 1.038e+00 G_adv_feat: 5.697e+00 D_true: 7.009e-01 D_false: 6.477e-01 : 100%|█████████▉| 400/401 [09:14<00:02,  2.19s/it]

Saved to ./checkpoints/MaskGAN/epoch_33_2023-04-25 20:47.pt


G_vgg: 4.646e-01 G_adv: 1.037e+00 G_adv_feat: 5.703e+00 D_true: 6.995e-01 D_false: 6.491e-01 : 100%|██████████| 401/401 [09:18<00:00,  1.39s/it]
  0%|          | 0/401 [00:00<?, ?it/s]

Saved to ./checkpoints/MaskGAN/epoch_33_2023-04-25 20:47.pt
Training epoch 34...


G_vgg: 4.651e-01 G_adv: 1.049e+00 G_adv_feat: 5.646e+00 D_true: 7.149e-01 D_false: 6.522e-01 :  86%|████████▋ | 346/401 [07:57<01:15,  1.37s/it]

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!free -m